## SINIESTROS VIALES EN BOGOTÁ

## Introducción:

El presente trabajo, tiene como finalidad, explorar el contexto del conjunto de datos seleccionado, el cual está basado en la recolección de información sobre siniestros viales en Bogotá, los cuales se presentaron el las localidades pertenecientes a la ciudad. Los datos de accidentes se recopilaron desde enero de 2015 hasta diciembre de 2019. Actualmente el dataset contempla 173.400 registros con 21 variables.


Dataset de Datos Abiertos Bogotá : Accidentes Viales Bogotá


## Enmarcar la preocupación:

* ¿Cuales son las circunstancias y factores que influyen en la ocurrencia de accidentes de tránsito?
* Índice de mortalidad a causa de accidentes automovilísticos.
* Factores de riesgo para aseguramiento de vehículos.
* Estado de la infraestructura vial (carreteras, señalización, geología)
* Influencia de las normas de tránsito (seguridad asociada al vehículo, normas de tránsito)
* Pericia en conducción de vehículos (¿El conductor está capacitado para conducir un vehículo?) 


## Identificación de interesados


-	Ciudadanos (Peatones, Conductores, Acompañantes)
-	Autoridades de tránsito
-            Gobernantes
-	Empresas Aseguradoras





## Problema, Necesidad, Oportunidad, Deseo, Objetivo: Pregunta inicial

* **Problema**

¿ Cual es el grado de afectación asociado a los siniestros viales, así como los factores que contribuyen a que se presente, identificando el impacto en la movilidad de la población?.

* **Necesidad**

¿Cómo se puede mitigar la ocurrencia de accidentes de tránsito en la ciudad con respecto a los factores que contribuyen a su generación?

* **Oportunidad**

Partir de la información existente en la plataforma de datos abierto para poder realizar un análisis de información que permita profundizar en la necesidad de tomar determinadas decisiones con respecto a los factores que influyen en la ocurrencia de los siniestros viales.

* **Deseo**

Mediante la utilización de los datos de la alcaldía se quiere identificar los factores preponderantes que influyen en la generación de accidentes de tránsito, que permita realizar planes de acción preventivos para la mitigación del riesgo en torno a los siniestros viales de la ciudad y bajar la tasa de ocurrencia.

## Objetivo

* Identificar cuales son los factores principales  que incrementan el riesgo de siniestros viales en la ciudad.

## Análisis de porqué, qué y para que

El objetivo del estudio permitirá establecer bajo qué condiciones y variables ocurren los accidentes de tránsito, identificar de qué manera se pueden mitigar y el impacto que tendría en la conservación de la vida de las personas.


## Objetivos de análisis:

**Obtener gráficas e indicadores  que permitan  la toma de decisiones, de acuerdo al análisis de la información de los accidentes, para contribuir a la reducción del problema.**
* Identificar las vías con mayor riesgo de accidentabilidad.(Mejorar infraestructura)
* Establecer las condiciones de riesgo para la toma de pólizas.
* Generar normas de tránsito que contribuyan a la mitigación del problema.

**Análisis de Tareas y las decisiones que se toman en el proceso**

* Establecer los factores de riesgo vial que contribuyen a la generación de accidentes de tránsito.
* Identificación de variables preponderantes en la ocurrencia de siniestros viales.
* Establecer relación de variables que influyen en el incremento de accidentes de tránsito.
* Normas que se deben modificar o adicionar a la reglamentación de tránsito.
* Mejoras en la infraestructura vial.
* Cobertura de amparos a nivel de pólizas de seguros de vehículos.

Preguntas iniciales de análisis que pueden apoyar el objetivo y las decisiones: Descriptivas, Diagnósticas, Predictivas. 

**1.6.1. Descriptivas:**

* ¿Por qué se generan los accidentes de tránsito?
* ¿Qué factores son determinantes para que ocurra un siniestro vial?
* ¿Qué tan conveniente es o no asegurar un vehículo?
* ¿Cual es la localidad en donde más se presentan siniestros?
* ¿Cual es la condición de los ciudadanos involucrados en accidentes de tránsito?
* ¿Qué tan probable es que ocurra un siniestro vial bajo determinadas condiciones?
* ¿Cuáles medidas se deberían reglamentar en la normatividad de tránsito para reducir los índices de accidentes?
* ¿Cual es la franja en donde se presentan los siniestros?

**1.6.2. Diagnósticas:**

* ¿Cómo disminuir la ocurrencia de siniestros viales en la ciudad para reducir los índices de accidentalidad?
* ¿Cómo incrementar la seguridad vial para proteger a los ciudadanos?
* ¿Cómo formular normas para mitigar la cantidad de siniestros?
* ¿Tiene alguna relación el género y la edad en la ocurrencia de siniestros?
* ¿Tiene alguna relación la clase y servicio del vehículo en la ocurrencia de siniestros?

**1.6.3. Predictivas:**

* ¿Los factores identificados influyen en la disminución de siniestros viales?
* ¿El análisis realizado permitirá identificar los riesgos a los que están expuestos los ciudadanos con respecto a siniestros viales?
* ¿Debe restringirse la circulación de alguna clase de vehículo en determinada franja de horario de circulación?


## Curar los datos

* Identificar Variables y su Metadata


Adicionalmente, el conjunto de datos encontrado, contempla los siguientes atributos:

# Importar Librerias

In [1]:
import pandas as pd # procesar datos, archivo CSV I/O (e.g. pd.read_csv)
import numpy as np  # linear algebra
import statistics 
import math
import matplotlib.pyplot as plt 
import seaborn as sns; sns.set_theme()
import scipy.stats as stats

from pandas_profiling import ProfileReport
from pandas_profiling.utils.cache import cache_file
#from darts import TimeSeries
%matplotlib inline

In [ ]:
from pycaret.datasets import get_data
data = get_data('insurance')
from pycaret.regression import *
reg1 = setup(data, target = 'charges', session_id=123, log_experiment=True, experiment_name='insurance1')

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:21:10
Status,. . . . . . . . . . . . . . . . . .,Preprocessing Data


Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
age,Numeric
sex,Categorical
bmi,Numeric
children,Categorical
smoker,Categorical
region,Categorical
charges,Label


# Limpieza y cargar de datos

Carga de archivo usando Pandas

In [ ]:
file_name = cache_file(
    "datasetSiniestro2.csv",
    "https://raw.githubusercontent.com/jgcotta/bigdata/master/datasetSiniestro.csv",
)

#If we need date
df = pd.read_csv(file_name, sep=';',parse_dates=[9])

df.dtypes
df.head(5)

In [ ]:
df.info()

In [ ]:
df.describe()

Convertir variables numéricas a Categoricas

In [ ]:
#df[['EDAD', 'LOCALIDAD', 'TOTAL_MUERTOS', 'TOTAL_HERIDOS']].apply(lambda x: x.astype('category'))

#cols = ['EDAD', 'HORA_NUM', 'TOTAL_MUERTOS', 'TOTAL_HERIDOS']

#df['EDAD'] = df['EDAD'].astype('category')
#df['LOCALIDAD'] = df['LOCALIDAD'].astype('category')


df.info();

# Correlación Lineal (Variables Numéricas)

In [ ]:
df.corr();

In [ ]:
sns.heatmap(df.corr(),data=df,annot=True)

In [ ]:
d=df.loc[:, ['EDAD', 'TOTAL_HERIDOS', 'SEXO', 'TOTAL_MUERTOS']]

In [ ]:
dx = d.corr(method ='pearson')
dx

In [ ]:
fig, ax = plt.subplots(figsize = ( 5 , 3 ))

sns.scatterplot(ax = ax, x="EDAD", y="TOTAL_HERIDOS", data=df);
# Set label for x-axis
ax.set_xlabel( "Edad" , size = 12 )
  
# Set label for y-axis
ax.set_ylabel( "Total Heridos" , size = 12 )
  
# Set title for plot
ax.set_title( "CORRELACIÓN EDAD - TOTAL HERIDOS" , size = 20 )
plt.show()

Quitar duplicados.

In [ ]:
drop_duplicates=df.drop_duplicates(subset=['CODIGO_SINIESTRO'])

# Análisis exploratorio variables categóricas

Histograma de Edad de implicados en accidentes

In [ ]:
drop_duplicates.hist(column='EDAD',    # Column to plot
                   figsize=(9,6),   # Plot size
                   bins=20) 

Agrupación de la población por actor víal

In [ ]:
tf2 = pd.crosstab(index=drop_duplicates["ACTOR_VIAL"],columns="Cantidad") 
tf2

Se obtiene la proporción del tipo de personas involucradas.

In [ ]:
drop_duplicates.ACTOR_VIAL.value_counts().plot(kind='pie', ylabel = "", title="ACTOR VIAL")

In [ ]:
tf2/tf2.sum()

In [ ]:
dsexo = (drop_duplicates.groupby("SEXO").agg(Frecuencia=("SEXO", "count")).head())
dsexo

In [ ]:
drop_duplicates.SEXO.value_counts().plot(kind='pie', ylabel = "", title="SEXO")

# PREGUNTAS DESCRIPTIVAS

1. ¿Cual es el día de la semana con mayor número de accidentes del periodo entre 2015 al 2019?

La día con más accidentados es el "VIERNES"

In [ ]:
tf3 = pd.crosstab(index=drop_duplicates["DIA"], 
                     columns="Cantidad") 
tf3

In [ ]:
df.DIA.value_counts().plot(kind='pie', ylabel = "", title="DIA")

In [ ]:
#tf3.plot.line(stacked=True)

line,ax = plt.subplots(figsize=(10,4))
# Set title for plot
ax.set_title("Accidentes Por Día", fontsize=20)
# Set label for y-axis
ax.set_xlabel( "Día" , size = 12 )
sns.lineplot(x = "DIA", y = "Cantidad", data = tf3)
plt.show()


2. ¿Cual es la localidad con mayor número de accidentes del periodo entre 2015 al 2019?

La tendencia indica que la localidad con mayor número de accidentes para el periodo es "KENNEDY"

In [ ]:
#df.LOCALIDAD
localidades = drop_duplicates.LOCALIDAD.unique()
len(localidades)

In [ ]:
localidades_por_accidente= drop_duplicates.LOCALIDAD.value_counts()
localidades_por_accidente

In [ ]:
localidades_por_accidente.plot(kind='barh')

3. ¿Cual es la hora con mayor indice accidentalidad del periodo entre 2015 al 2019?

In [ ]:
dhora = pd.crosstab(index=drop_duplicates["HORA_NUM"], 
                     columns="Cantidad") 

In [ ]:
maximo = dhora[dhora.Cantidad == dhora.Cantidad.max()]
maximo

In [ ]:
line,ax = plt.subplots(figsize=(20,5))
# Set title for plot
ax.set_title("Accidentes Por Hora", fontsize=20)
# Set label for y-axis
ax.set_xlabel( "Hora" , size = 12 )
sns.lineplot(x = "HORA_NUM", y = "Cantidad", data = dhora)
plt.show()

# Series de Tiempo

4. ¿Cúal fue la fecha con mayor indice de accidentalidad del año 2019?

In [ ]:
start_date = "2019-1-1"
end_date = "2019-12-31"
after_start_date = drop_duplicates["FECHA"] >= start_date
before_end_date = drop_duplicates["FECHA"] <= end_date
between_two_dates = after_start_date & before_end_date
filtered_dates = drop_duplicates.loc[between_two_dates]
tf4 = pd.crosstab(index=filtered_dates["FECHA"], columns="Cantidad") 
maximo = tf4[tf4.Cantidad == tf4.Cantidad.max()]
maximo

In [ ]:
line,ax = plt.subplots(figsize=(20,5))
ax.set_title("Accidentes Por Fecha", fontsize=20)
sns.lineplot(x = "FECHA", y = "Cantidad", data = tf4)
plt.show()

# HIPOTESIS DIAGNÓSTICAS

**Hipótesis Diagnóstica 1**

\begin{align}
       H_{0} : \; El\;género\;NO\;se\;relaciona\;con\;el\;tipo\;de\;afectación\;del\;individuo\;en\;un\;accidente \\
       H_{A} : \; El\;género\;se\;relaciona\;con\;el\;tipo\;de\;afectación\;del\;individuo\;en\;un\;accidente\\
\end{align}

Se halla tabla de contingencia

In [ ]:
tabla_contingenciah1=pd.crosstab(df["SEXO"],df["ESTADO"])
print('Tabla Contingencia :-\n',tabla_contingenciah1)

In [ ]:
#Valores Observados
valores_observados= tabla_contingenciah1.values 
print("Valores Observados :-\n",valores_observados)

In [ ]:
#Valores esperados
import scipy.stats
b=scipy.stats.chi2_contingency(valores_observados)
valores_esperados = b[3]
print("Valores esperados :\n",valores_esperados)

In [ ]:
#Grados de Libertad
numero_filas=len(tabla_contingenciah1.iloc[0:2,0])
numero_columnas=len(tabla_contingenciah1.iloc[0,0:3])
gradoslibertad=(numero_filas-1)*(numero_columnas-1)
print("Grados de Libertad:",gradoslibertad)

In [ ]:
#Nivel Significancia 5%
alpha=0.05

In [ ]:
#chi-square statistic - χ2
from scipy.stats import chi2
chi_square=sum([(o-e)**2./e for o,e in zip(valores_observados,valores_esperados)])
chi_square_statistic=chi_square[0]+chi_square[1]
print("chi-square statistic:",chi_square_statistic)

In [ ]:
#valor critico
critical_value=chi2.ppf(q=1-alpha,df=gradoslibertad)
print('critical_value:',critical_value)

In [ ]:
#p-valor
p_value=1-chi2.cdf(x=chi_square_statistic,df=gradoslibertad)
print('p-value:',p_value)

In [ ]:
#Resumen de valores obtenidos
print('Nivel Significancia: ',alpha)
print('Grados de Libertad: ',gradoslibertad)
print('Estadístico Chi Cuadrado:',chi_square_statistic)
print('Valor Crítico:',critical_value)
print('p-valor:',p_value)

In [ ]:
#Conclusión
if chi_square_statistic>=critical_value:
    print("Se rechaza la hipótesis nula H0, por tanto existe relación entre el género y la gravedad del sujeto en el accidente")
else:
    print("Se acepta la hipótesis nula H0, por tanto no existe relación entre el género y la gravedad del sujeto en el accidente")
#También se puede comparar el p-valor con el valor de significancia

**Hipótesis Diagnóstica 2**

 \begin{align*}
       H_{0} : \; El\;tipo\;de\;vehículo\;\textbf{NO}\;influye\;en\;la\;gravedad\;del\;accidente \\
       H_{A} : \; El\;tipo\;de\;vehículo\;\textbf{SÍ}\;influye\;en\;la\;gravedad\;del\;accidente\\
    \end{align*}

In [ ]:
tabla_contingenciah2=pd.crosstab(drop_duplicates.TIPO_AUTOMOVIL, drop_duplicates.TIPO_ACCIDENTE, margins=True)
tabla_contingenciah2

In [ ]:
stats.chi2_contingency(observed = tabla_contingenciah2)

In [ ]:
#Valores Retornados:
print('Grados de Libertad: ',98)
print('Estadístico Chi Cuadrado:',42494.11133322942)
print('p-valor:',0.0)


Basado en el p-valor que es cercano a 0, se puede concluir que se rechaza la hipotesis nula, por lo tanto existe relación entre el tipo de vehículo y la gravedad del accidente.

# TABLA DE FRECUENCIA

In [ ]:
#Formart to percentage
format_dict = {'CONDUCTOR': '{:.2%}','PASAJERO': '{:.2%}','CICLISTA': '{:.2%}','MOTOCICLISTA': '{:.2%}','FILA_TOTAL': '{:.2%}'}
genero_actor = pd.crosstab(index=df["SEXO"], 
                           columns=df["ACTOR_VIAL"],margins=True)
genero_actor.columns = ["CONDUCTOR","PASAJERO","CICLISTA","MOTOCICLISTA","FILA_TOTAL"]
genero_actor.index= ["MASCULINO","FEMENINO","COL_TOTAL"]
genero_actor

Tabla de frecuencias Relativas: Relación entre Actor_vial y genéro implicado en sinisetro (En porcentaje), El 77.40% de los implicados en accidentes, son mujeres pasajeros de algún servicio público.

In [ ]:
tabla_relativa=genero_actor/genero_actor.loc["COL_TOTAL","FILA_TOTAL"]
tabla_relativa.style.format(format_dict)

Frecuencias Condicionadas(Proporciones) por columnas(Cada valor/total de la condicion de la columna). Ejemplo: Entre el año 2015 al 2019 de los conductores de automóviles implicados en accidentes, el 82.64% son mujeres. 

In [ ]:
condicionada1=genero_actor/genero_actor.loc["COL_TOTAL"]
condicionada1[["CONDUCTOR","PASAJERO","CICLISTA","MOTOCICLISTA"]].style.format(format_dict)

Frecuencias Condicionadas(Proporciones) por Filas(Cada valor/total de la condicion de la fila). Ejemplo: Del 2015 al 2019 de los siniestros de tránsito en Bogotá, de la población masculina el 32.49% de los accidentados fueron motociclistas.

In [ ]:
condicionada2=genero_actor.div(genero_actor["FILA_TOTAL"],
                   axis=0)
condicionada2.loc[["MASCULINO","FEMENINO"]].style.format(format_dict)